Tema2
Lidia Istrate
342C4


Functions for reading the input:

In [19]:
import re


#Representation
CONSTANT = "CONSTANT"
VARIABLE = "VARIABLE"
FUNCTION = "FUNCTION"
ATOM = "ATOM"
CONDITION = "CONDITION"
RULE = "RULE"
QUESTION = "QUESTION"
ANSWER = "ANSWER"
COMMENT = "COMMENT"
NOT = "NOT"
OR = "OR"
AND = "AND"




def make_const(value):
    return (CONSTANT, value)


def make_var(name):
    return (VARIABLE, name)


def make_function_call(function, args):
    return (FUNCTION, function, args)


def make_atom(predicate, args):
    return (ATOM, predicate, args)


def make_question(predicate, args):
    return (ATOM, predicate, args)


def make_condition(conditions):
    return (CONDITION, CONDITION, conditions)

def make_rule(atom, conditions):
    
    rule = []
    for c in conditions:
        rule.append(make_neg(c))
    rule.append(atom)
    
    return (OR, OR, rule)



def make_neg(sentence):
    return (NOT, NOT, sentence)

def make_or(sentences):
    return (OR, OR, sentences) 

def is_constant(f):
    return f[0] == CONSTANT


def is_variable(f):
    return f[0] == VARIABLE


def is_function_call(f):
    return f[0] == FUNCTION


def is_atom(f):
    return f[0] == ATOM


def is_question(f):
    return f[0] == QUESTION


def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)

def is_sentence(f):
    return is_atom(f) or f[0] == CONDITION or f[0] == OR or f[0] == NOT


def has_args(f):
    return is_function_call(f) or is_sentence(f)


def get_value(f):
    if is_constant(f):
        return f[1]
    return None


def get_name(f):
    if is_variable(f):
        return f[1]
    return None


def get_head(f):
    if is_function_call(f) or is_atom(f):
        return f[1]
    if f[0] == CONDITION:
        return CONDITION
    if f[0] == OR:
        return OR
    if f[0] == NOT:
        return NOT
    return None


def get_args(f):
    if has_args(f):
        return f[2]
    return []

def replace_args(formula, new_args):
    return (formula[0], formula[1], new_args)

def get_type(line):
    
    if line[0] == "?":
        return QUESTION
    elif ":" in line and line[0] != ":":
        return RULE
    elif line[0] == ":":
        return ANSWER
    elif line[0] == "#":
        return COMMENT
    else:
        return ATOM
        

facts = []
rules = []
questions = []

def process_atom(line):
        
    predicate = ''
    current_string = ''
    args = []
    index = 0
    i = 0
    while i < len(line):
        
        if line[i] == '(' and predicate == '':
            predicate = current_string

            current_string = ''
            index = i+1
        
        elif line[i] == ',' or line[i] == ')':
            if current_string != '':
                if '?' in current_string:
                    var = make_var(current_string)
                    args.append(var)

                else:
                    const = make_const(current_string)
                    args.append(const)

                
            current_string = ''
        
        elif line[i] == ' ':
            index = i+1
        elif line[i] == '(' and predicate != '':
            j = index
            function_string = ''
            
            count = 0
            first_paranth = False
            ok = False
            
            while ok == False:
                if line[j] == ')':
                    count -= 1
                    if first_paranth == True and count == 0:
                        ok = True
                if line[j] == '(':
                    count += 1
                    first_paranth = True

                function_string += line[j]
                j = j + 1
            

            atom_f = process_atom(function_string)
            function = make_function_call(atom_f[1], atom_f[2])
            args.append(function)
            current_string = ''
            i = j-1

                 
        else:
            current_string = current_string + line[i]
        i = i + 1
            

    return make_atom(predicate, args)
    
    
def process_rule(line):
    conditions = []
    i = 0
    
    current_string = ''
    while (line[i] + line[i+1] != ' :'):
        current_string = current_string + line[i]
        i = i + 1
    atom = process_atom(current_string)
    i = i + 3

    
    while i < len(line):
        current_string = ''
        count = 0
        first_paranth = False
        ok = False

        while ok == False:
            if line[i] == ')':
                count -= 1
                if first_paranth == True and count == 0:
                    ok = True
            if line[i] == '(':
                count += 1
                first_paranth = True

            current_string += line[i]
            i = i + 1    

        condition = process_atom(current_string)
        conditions.append(condition)

        
        i = i + 2
    
    return make_rule(atom, conditions)
    
    
def read_input(filename):
    
    f = open(filename, "r")
    for line in f:
        if line and line != '\n':
            full_line = line
            line = line.split()
            

            line_type = get_type(line)
            
            if line_type == ATOM:
                atom = process_atom(full_line)
                facts.append(atom)
            elif line_type == QUESTION:
                q_line = full_line[2:len(full_line)]
                atom = process_atom(q_line)
                question = make_question(atom[1], atom[2])
                questions.append(question)
            elif line_type == RULE:
                rule = process_rule(full_line)
                rules.append(rule)            
                
    f.close()


def pFail(message, f):
    print(message + " <" + str(f) + ">")
    return False
def check_term(T):
    if is_constant(T):
        return ((get_value(T) is not None) or pFail("The value of the constant is None", T))
    if is_variable(T):
        return ((get_name(T) is not None) or pFail("The name of the variable is None", T))
    if is_function_call(T):
        return (not [t for t in get_args(T) if not check_term(t)] and       
    (get_head(T) is not None or pFail("Function is not callable", T)))
    return pFail("Term is not one of constant, variable or function call", T)
def check_atom(A):
    if is_atom(A):
        return (not [t for t in get_args(A) if not check_term(t)] and     
    (get_head(A) is not None or pFail("Predicate name is None", A)))
    return pFail("Is not an atom", A)
def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [CONDITION, OR]:
            return ((len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S))    
        and not [s for s in get_args(S) if not check_sentence(s)])
        if get_head(S) == NOT:
            #return ((len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) 
            return True
        #and check_sentence(get_args(S)[0]))
    return pFail("Not sentence or unknown type", S)

def add_statement(kb, statement):
    
    if check_sentence(statement):
        kb.append(statement)
        print("OK: Added statement " + print_formula(statement, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(statement, True)+"><" + str(statement) + ">")
    return False
    
    
            
def is_positive_literal(L):
    return is_atom(L)
def is_negative_literal(L):
    return get_head(L) == NOT #and is_positive_literal(get_args(L)[0])
def is_literal(L):
    return is_positive_literal(L) or is_negative_literal(L)            




def get_premises(formula):
    premises = []
    if is_rule(formula):
        args = get_args(formula) 
        for i in args:
            if get_head(i) == NOT:
                premises.append(get_args(i))
    return premises

def get_conclusion(formula):
    if is_rule(formula):
        args = get_args(formula)
        return args[len(args) - 1]
    return None

def is_fact(formula):
    return is_positive_literal(formula)

def is_rule(formula):
    args = get_args(formula)
    for i in args:
        if is_negative_literal(i):
            return True
    return False


def equal_terms(t1, t2):
    if is_constant(t1) and is_constant(t2):
        return get_value(t1) == get_value(t2)
    if is_variable(t1) and is_variable(t2):
            return get_name(t1) == get_name(t2)
    if is_function_call(t1) and is_function_call(t2):
        if get_head(t1) != get_head(t2):
            return all([equal_terms(get_args(t1)[i], get_args(t2)[i]) for i in range(len(get_args(t1)))])
    return False

def is_equal_to(a1, a2):
    if not (is_atom(a1) and is_atom(a2) and get_head(a1) == get_head(a2) and len(get_args(a1)) == len(get_args(a2))):
        return False
    return all([equal_terms(get_args(a1)[i], get_args(a2)[i]) for i in range(len(get_args(a1)))])


def substitute(f, substitution):
    if substitution is None:
        return None
    if is_variable(f) and (get_name(f) in substitution):
        return substitute(substitution[get_name(f)], substitution)
    if has_args(f):
        return replace_args(f, [substitute(arg, substitution) for arg in get_args(f)])
    return f

def occur_check(v, t, subst):
    if v == t:
        return True
    if is_variable(t) and (get_name(t) in subst):
        return occur_check(v, substitute(t, subst), subst)
    if has_args(t):
        return any(occur_check(v, arg, subst) for arg in get_args(t))
        
    return False


def unify(f1, f2, subst = None):
    if subst is None:
        subst = {}
    
    stack = [(f1, f2)]
    
    while len(stack) > 0:
        (s, t) = stack.pop()
        
        while is_variable(s) and (get_name(s) in subst):
            s = substitute(s, subst)
        while is_variable(t) and (get_name(t) in subst):
            t = substitute(t, subst)
        if s != t:
            if is_variable(s):
                if occur_check(s, t, subst):
                    return False
                else:
                    subst[get_name(s)] = t
            elif is_variable(t):
                if occur_check(t, s, subst):
                    return False
                else:
                    subst[get_name(t)] = s
            elif has_args(s) and has_args(t) and len(get_args(s)) == len(get_args(t)):
                if get_head(s) == get_head(t):
                    for i in range(0, len(get_args(s))):
                        stack.append((get_args(s)[i], get_args(t)[i]))
                else:
                    return False
            else:
                return False
                 
    return subst


from copy import deepcopy

def apply_rule(rule, facts):
    premises = get_premises(rule)
    conclusion = get_conclusion(rule)
    ok_substs = [{}]
    facts_from_rule = []
    
    for premise in premises:
        intermediate_substs = []
        
        for ok_subst in ok_substs:
            for fact in facts:
                unify_result = unify(premise, fact, deepcopy(ok_subst))
                
                if unify_result != False:
                    intermediate_substs.append(unify_result)       
        ok_substs = intermediate_substs
        
    
    for substitution in ok_substs:
        facts_from_rule.append(substitute(conclusion, substitution))
    
    return facts_from_rule



def get_kb(rules, facts):
    kb = []
    
    for fact in facts:
        add_statement(kb, fact)

    for rule in rules:
        add_statement(kb, rule)
        
    return kb

def print_formula(f, return_result = False):
    ret = ""
    if is_term(f):
        if is_constant(f):
            ret += str(get_value(f))
        elif is_variable(f):
            ret += "?" + get_name(f)
        elif is_function_call(f):
            ret += str(get_head(f)) + "[" + "".join([print_formula(arg, True) + "," for arg in get_args(f)])[:-1] + "]"
        else:
            ret += "???"
    elif is_atom(f):
        ret += str(get_head(f)) + "(" + "".join([print_formula(arg, True) + ", " for arg in get_args(f)])[:-2] + ")"
    elif is_sentence(f):
        # negation, conjunction or disjunction
        args = get_args(f)
        if len(args) == 1:
            ret += str(get_head(f)) + print_formula(args[0], True)
        else:
            ret += "(" + str(get_head(f)) + "".join([" " + print_formula(arg, True) for arg in get_args(f)]) + ")"
    else:
        ret += "???"
    if return_result:
        return ret
    print(ret)
    return

def forward_chaining(kb, theorem, verbose = True):
    # Salvăm baza de date originală, lucrăm cu o copie
    local_kb = deepcopy(kb)
    # Două variabile care descriu starea căutării
    got_new_facts = True   # s-au găsit fapte noi la ultima căutare
    is_proved = False      # a fost demostrată teorema
    # Verificăm dacă teorema este deja demonstrată
    for fact in filter(is_fact, local_kb):
        if unify(fact, theorem):
            print(unify(fact, theorem))
            if verbose: print("This already in KB: " + print_formula(fact, True))
            is_proved = True
            #break
    while (not is_proved) and got_new_facts:
        got_new_facts = False
        for rule in filter(is_rule, local_kb):
            # Pentru fiecare regulă
            new_facts = apply_rule(rule, list(filter(is_fact, local_kb)))
            new_facts = list(filter(lambda fact: not any(list(filter(lambda orig: is_equal_to(fact, orig), local_kb))), new_facts))
            if new_facts:
                if verbose: print("Applied rule: " + print_formula(rule, True) + ", obtained " + str(len(new_facts)) + " new facts.")
                if any(filter(lambda t: is_variable(t), get_args(get_conclusion(rule)))) and \
                    any(filter(lambda fact: is_equal_to(fact, get_conclusion(rule)), new_facts)):
                        print("Demonstration is too general, the conclusion is not instantiated (facts obtained:", 
                              ",".join([print_formula(f, True) for f in new_facts]),").")
                        return False
                got_new_facts = True
                for fact in new_facts:
                    #if verbose: print("New fact: " + print_formula(fact, True))
                    if unify(fact, theorem) != False:
                        is_proved = True
                        add_statement(local_kb, fact)
                        if verbose: print("Now in KB: " + print_formula(fact, True))
                        #break
                    add_statement(local_kb, fact)
            #if is_proved:
            #    break
    if verbose:
        if is_proved:
            print("The theorem is TRUE!")
        else:
            print("The theorem is FALSE!")
    return is_proved


def has_variables(theorem):
    params = theorem[2]
    
    vars_theorem = filter(is_variable, params)
    
    if len(vars_theorem) > 0:
        return True
    return False


def backward_chaining(kb, theorem, verbose):
    #local_kb = deepcopy(kb)
    is_proved = False
    
    var_dict = {}
    unification_list = []
    
    
    for fact in filter(is_fact, kb):
        #print(theorem)
        if unify(fact, theorem) != False:
            #print(unify(fact, theorem))
            unification_list.append(unify(fact, theorem))
            if verbose: print("This already in KB: " + print_formula(fact, True))
            is_proved = True
            #break
            
    if is_proved:
        return (is_proved, unification_list)
            
    check = False
    
    save_check = check
    temp_unification_list = []
    
    for rule in filter(is_rule, kb):
        conclusion = get_conclusion(rule)
        save_check = check or save_check

        if get_head(conclusion) == get_head(theorem) and len(get_args(conclusion)) == len(get_args(theorem)):

            dict_var_var = {}
            dict_var_val = {}
            
            conclusion_args = get_args(conclusion)
            theorem_args = get_args(theorem)
            
            for i in range(0, len(conclusion_args)):
                dict_var_var[conclusion_args[i]] = theorem_args[i]
            
            premises = get_premises(rule)

            check = True
            temp_unification_list = []
            for premise in premises:
                
                args_premise = get_args(premise)
                for j in range (0, len(args_premise)):
                    if args_premise[j] in dict_var_var:
                        print("before ", args_premise[j])
                        args_premise[j] = dict_var_var[args_premise[j]]
                        print("after ", args_premise[j])
                
                #premise[2] = args_premise
                #print(len(premise))
                #premise = (premise[0], premise[1], args_premise)
                
                (verif, subst) = backward_chaining(kb, premise, verbose)
                check = check and verif

                temp_unification_list.append(subst)

            if check == True:
                for i in temp_unification_list:
                    unification_list.append(i)

    save_check = check or save_check
    return (save_check, unification_list)


def main():
    read_input("teste/test1.txt")
    #for fact in facts:
        #print(fact)
        
    #for rule in rules:
        #print(rule)
        #print(is_rule(rule))
        
    #for question in questions:
        #print(question)     
        
    kb = get_kb(rules, facts)
        
        
    local_kb = deepcopy(kb)
    (check, unif) = backward_chaining(local_kb, questions[-6], True)
    print(check)
    print(unif)
    
    #print(substitute(questions[-2], [('?a': ('CONSTANT', '1')), ('?b': ('CONSTANT', '3'))]))
    
    
    

main()

OK: Added statement P(1)
OK: Added statement P(2)
OK: Added statement Q(3)
OK: Added statement t(??x)
OK: Added statement (OR (NOT ??? ??? ???) (NOT ??? ??? ???) r(??a))
OK: Added statement (OR (NOT ??? ??? ???) (NOT ??? ??? ???) s())
OK: Added statement (OR (NOT ??? ??? ???) (NOT ??? ??? ???) s())
OK: Added statement (OR (NOT ??? ??? ???) (NOT ??? ??? ???) pq(??x, ??y))
before  ('VARIABLE', '?x')
after  ('CONSTANT', '2')
This already in KB: P(2)
before  ('VARIABLE', '?y')
after  ('CONSTANT', '3')
This already in KB: Q(3)
True
[[{}], [{}]]
